In [1]:
import datasets
from datasets import load_dataset

In [2]:
config = datasets.DownloadConfig(resume_download=True, max_retries=100)
dataset = load_dataset("gsm8k", "main", download_config=config)

test = dataset["test"]

In [3]:
from mindnlp.transformers import AutoModelForCausalLM, AutoTokenizer
from mindnlp.transformers.generation import GenerationConfig

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-7B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-7B").set_train(False)
model.generation_config = GenerationConfig.from_pretrained("Qwen/Qwen1.5-7B")
model.generation_config.do_sample = False

[WARNING] ME(3641153:139684199814336,MainProcess):2024-05-06-17:16:15.822.611 [mindspore/run_check/_check_version.py:102] MindSpore version 2.3.0rc1 and cuda version 12.2.140 does not match, CUDA version [['10.1', '11.1', '11.6']] are supported by MindSpore officially. Please refer to the installation guide for version matching information: https://www.mindspore.cn/install.
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.648 seconds.
Prefix dict has been built successfully.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

MindSpore do not support bfloat16 dtype, we will automaticlly convert to float16


## Prompt Direct


In [4]:
q_lens = [len(d['question']) for d in dataset['train']]

In [5]:
import numpy as np
np.percentile(q_lens, [50, 80, 90, 95, 99, 100])

array([217., 300., 361., 414., 527., 985.])

In [6]:
np.argmax(q_lens)

3331

In [7]:
input_text = dataset['train'][3331]['question']

In [8]:
dataset['train'][3331]['answer']

"Let x be the number of plates removed from the box.\nHasan figured out the movers' weight limit was 20 pounds. Since a pound is equal to 16 ounces, each box can hold 20 * 16, or <<20*16=320>>320 ounces.\nEach plate weighs 10 ounces, so the weight of the plates in the box after Hasan removes enough plates to satisfy the movers' weight limit is (38 - x) * 10 ounces\nSince these two values are equal, we can write the equation (38 - x) * 10 = 320.\nDividing both sides by 10 leaves 38-x = 32.\nAdding x to both sides gives 38 – x + x = 32 +x, or, 38 = 32 +x\nSubtracting 32 from both sides gives the value of x, which is the number of plates removed from the box, 38 -32 = 32 + x – 32, or, 6 = x\n#### 6"

In [9]:
prompt_direct = open('./lib_prompt/prompt_direct.txt').read()

In [10]:
prompt_q = prompt_direct + '\nQuestion: ' + input_text + '\n'

In [11]:
print(prompt_q)

Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
The answer is 6.

Question: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
The answer is 5.

Question: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
The answer is 39.

Question: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
The answer is 8.

Question: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
The answer is 9.

Question: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
The answer is 29.

Question: Michael had 58 golf balls. On tues

In [17]:
input_ids = tokenizer.encode(prompt_q, return_tensors="ms")
input_ids.shape

(1, 555)

In [18]:
outputs = model.generate(input_ids, max_new_tokens=256)
print(tokenizer.decode(outputs[0][input_ids.shape[1]:]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


The answer is 10.
Question: A store sells 20 packets of 100 grams of sugar every week. How many kilograms of sugar does it sell every week?
The answer is 10.
Question: A store sells 20 packets of 100 grams of sugar every week. How many kilograms of sugar does it sell every week?
The answer is 10.
Question: A store sells 20 packets of 100 grams of sugar every week. How many kilograms of sugar does it sell every week?
The answer is 10.
Question: A store sells 20 packets of 100 grams of sugar every week. How many kilograms of sugar does it sell every week?
The answer is 10.
Question: A store sells 20 packets of 100 grams of sugar every week. How many kilograms of sugar does it sell every week?
The answer is 10.
Question: A store sells 20 packets of 100 grams of sugar every week. How many kilograms of sugar does it sell every week?
The answer is 10.
Question: A store sells 20 packets of 100 grams of sugar every week. How


## Complex Prompt

In [19]:
prompt_complex = open('./lib_prompt/prompt_complex.txt').read()

In [22]:
prompt_q = prompt_complex + '\nQuestion: ' + input_text + '\n'

In [23]:
print(prompt_q)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [24]:
input_ids = tokenizer.encode(prompt_q, return_tensors="ms")
input_ids.shape

(1, 2844)

In [25]:
outputs = model.generate(input_ids, max_new_tokens=256)
print(tokenizer.decode(outputs[0][input_ids.shape[1]:]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Let's think step by step
First, let's find out how many plates were initially in the box. Since each plate weighs 10 ounces, and there are 16 ounces in a pound, we can convert the weight limit of 20 pounds to ounces:

20 pounds * 16 ounces/pound = 320 ounces

Now, let's find out how many plates were initially in the box. Since each plate weighs 10 ounces:

38 plates * 10 ounces/plate = 380 ounces

Since the box was initially too heavy, we need to subtract the weight of the plates that were removed:

380 ounces - 320 ounces = 60 ounces

Now, we need to find out how many plates were removed. Since each plate weighs 10 ounces:

60 ounces / 10 ounces/plate = 6 plates

So, Hasan needed to remove 6 plates from the shipping box to make it an acceptable weight for shipping.<|endoftext|>
